### Import Required Libraries and Set Up Environment Variables

In [10]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [11]:
from pynytimes import NYTAPI

In [12]:
# Set environment variables from the .env in the local environment
load_dotenv()

NYT_API_KEY = os.getenv("NYT_API_KEY")
TMDB_API_KEY = os.getenv("TMDB_API_KEY")
 

### Access the New York Times API

In [13]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL


nyt_query_url = f"{url}&api-key={NYT_API_KEY}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}"

In [75]:
nyt_query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={NYT_API_KEY}"
print(nyt_query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=rLCifMAG808EQrFdkAm3ArLMEBcKUBGN


In [14]:
# Create an empty list to store the reviews
reviews_list=[]

# loop through pages 0-19
for i in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{nyt_query_url}&page={i}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
        # Print the page that was just retrieved
        print(f"checked page{i}")
    except:

        # Print the page number that had no results then break from the loop
        print(f"Page {i} had no results")
        break

checked page0
checked page1
checked page2
checked page3
checked page4
checked page5
checked page6
checked page7
checked page8
checked page9
checked page10
checked page11
checked page12
checked page13
checked page14
checked page15
checked page16
checked page17
checked page18
checked page19


In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [16]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [17]:
for review in reviews_list[:5]:
    print(json.dumps(review, indent=4))

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [18]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"] = reviews_list_df["keywords"].apply(extract_keywords)
reviews_list_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [20]:
reviews_list_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [21]:
#reviews_list_df.head()
reviews_list_df['headline.main']

0            ‘The Attachment Diaries’ Review: Love, Sick
1      Review: ‘What’s Love Got to Do With It?’ Proba...
2      ‘You Can Live Forever’ Review: Do You Love Me ...
3      ‘A Tourist’s Guide to Love’ Review: A Wearying...
4         ‘Other People’s Children’ Review: True Romance
                             ...                        
195    Review: A Combustible Pair Find Love in ‘The O...
196    Review: Love as the World Wars, in ‘The Ottoma...
197    Review: It’s All Mirth and Taxes in ‘Love & Ta...
198    Review: ‘Everybody Loves Somebody,’ a Rom-Com ...
199    Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...
Name: headline.main, Length: 200, dtype: object

In [23]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def extract_title(headline_main):
    # Check if headline starts with "Review:"
    if headline_main.startswith("Review:"):
        # Extract the title after "Review:"
        return headline_main.split("Review:")[1].strip()

    # Original pattern extraction
    start = headline_main.find("\u2018") + 1
    end = headline_main.find("\u2019 Review")
    if start > 0 and end > start:
        return headline_main[start:end]
    else:
        return "Title Not Found"

# Apply the function to the DataFrame
reviews_list_df['title'] = reviews_list_df['headline.main'].apply(extract_title)

In [24]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = (reviews_list_df["title"]).to_list()
titles

['The Attachment Diaries',
 '‘What’s Love Got to Do With It?’ Probably a Lot',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink C

### Access The Movie Database API

In [25]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + TMDB_API_KEY


movie_detail_url='https://api.themoviedb.org/3/movie/'
apikey = "api_key="+ TMDB_API_KEY

In [27]:
def moviedetails(responseJson):
    for movie in responseJson['results']:
        # Get movie id
        movieid=movie['id']
        # Make a request for a the full movie details
        movie_url_final=movie_detail_url+str(movieid)+"?"+apikey
        moviedetails=requests.get(movie_url_final)
        movieJson=moviedetails.json()
        # Extract the genre names into a list
        genre_dict=movieJson['genres']
        genre_names = [genre['name'] for genre in genre_dict]
        # Extract the spoken_languages' English name into a list       
        spoken_languages=movieJson['spoken_languages']       
        languages = [lang['english_name'] for lang in spoken_languages]
        # Extract the production_countries' name into a list
        production_countries=movieJson['production_countries']
        countries = [countries['name'] for countries in production_countries]
        # Add the relevant data to a dictionary and
        movieJson['spoken_languages']=languages
        movieJson['production_countries']=countries
        movieJson['genres']=genre_names
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(movieJson)
        #print(json.dumps(movieJson,indent=4))

In [29]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
        if request_counter % 50 == 0:
            time.sleep(1)

    # Add 1 to the request counter
        request_counter += 1

    # Perform a "GET" request for The Movie Database
        movie_reviews = requests.get(f"{url}{title}{tmdb_key_string}").json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        try:
        # Get movie id
                movie_id = movie_reviews["results"][0]["id"]

        # Make a request for a the full movie details
                full_movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}"

        # Execute "GET" request with url
                full_movie_details_query = requests.get(full_movie_details_url).json()
        
        # Extract the genre names into a list
                genres = []
                for genre in full_movie_details_query["genres"]:
                    genres.append(genre["name"])

        # Extract the spoken_languages' English name into a list
                spoken_languages = []
                for language in full_movie_details_query["spoken_languages"]:
                    spoken_languages.append(language["english_name"])

        # Extract the production_countries' name into a list
                production_countries = []
                for country in full_movie_details_query["production_countries"]:
                    production_countries.append(country["name"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
                combined_movie_data = {
                    "title" : full_movie_details_query["title"], 
                   "original_title" : full_movie_details_query["original_title"],
                   "budget" : full_movie_details_query["budget"] ,
                   "original_language" : full_movie_details_query["original_language"],
                   "homepage" : full_movie_details_query["homepage"], 
                   "overview" : full_movie_details_query["overview"], 
                   "popularity" : full_movie_details_query["popularity"], 
                   "runtime" : full_movie_details_query["runtime"], 
                   "revenue" : full_movie_details_query["revenue"], 
                   "release_date" : full_movie_details_query["release_date"], 
                   "vote_average" : full_movie_details_query["vote_average"], 
                   "vote_count" : full_movie_details_query["vote_count"],
                   "genres" : genres,
                   "spoken_languages" : spoken_languages,
                   "production_countries" : production_countries  
                }
                tmdb_movies_list.append(combined_movie_data)
        
        # Print out the title that was found
                print(f"{title} was found")

        except:
                print(f"{title} was not found")

The Attachment Diaries was found
‘What’s Love Got to Do With It?’ Probably a Lot was not found
You Can Live Forever was found
A Tourist’s Guide to Love was found
Other People’s Children was found
One True Loves was found
The Lost Weekend: A Love Story was found
A Thousand and One was found
Your Place or Mine was found
Love in the Time of Fentanyl was found
Pamela, a Love Story was found
In From the Side was found
After Love was found
Alcarràs was found
Nelly & Nadine was found
Lady Chatterley’s Lover was found
The Sound of Christmas was found
The Inspection was found
Bones and All was found
My Policeman was found
About Fate was found
Waiting for Bojangles was found
I Love My Dad was found
A Love Song was found
Alone Together was found
Art of Love was found
The Wheel was found
Thor: Love and Thunder was found
Both Sides of the Blade was found
Fire of Love was found
Love & Gelato was found
Stay Prayed Up was found
Benediction was found
Dinner in America was found
In a New York Minute was

In [30]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))
 

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.232,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [31]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)

In [32]:
# Convert the results to a DataFrame
movies_df=pd.DataFrame(tmdb_movies_list)
movies_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",37.074,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,42.461,100,37820,2023-04-07,6.520,76,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Sicilian Ghost Story,Sicilian Ghost Story,0,it,,In a little Sicilian village at the edge of a ...,9.305,122,0,2017-11-18,6.600,137,"[Mystery, Drama, Fantasy]",[Italian],"[France, Italy, Switzerland]"
137,The New Romantic,The New Romantic,0,en,,In order to escape her looming post-graduation...,24.663,82,0,2018-10-19,6.300,108,"[Drama, Comedy]",[English],[Canada]
138,Pimp,Pimp,0,en,,An urban love story set on the hard streets of...,4.933,86,0,2018-11-09,6.300,29,[Drama],[English],[United States of America]
139,In a Relationship,In a Relationship,0,en,,Long-term couple Owen and Hallie are breaking ...,12.717,92,0,2018-11-09,5.800,103,"[Drama, Comedy, Romance]",[English],"[Mexico, United States of America]"


In [33]:
# Create an empty list to store the reviews
reviews_list = []

# Function to fetch reviews from the New York Times API
def fetch_nyt_reviews(query_url, pages=20):
    reviews_list = []
    for page in range(pages):
        nyt_query_url_page = f"{query_url}&page={page}"
        response = requests.get(nyt_query_url_page)
        time.sleep(12)
        if response.status_code == 200:
            reviews = response.json()
            try:
                for review in reviews["response"]["docs"]:
                    reviews_list.append(review)
                print(f"Checked page {page}")
            except KeyError:
                print(f"Page {page} has no results")
                break
        else:
            print(f"Failed to retrieve page {page}, status code: {response.status_code}")
            break
    return reviews_list


In [35]:
# Convert the results to a DataFrame
tmdb_df=pd.DataFrame(tmdb_movies_list)

In [36]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df=pd.merge(reviews_list_df, tmdb_df, on='title', how='inner')

### Merge and Clean the Data for Export

In [37]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix=['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove=["[","]","'",]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column]=merged_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column]= merged_df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
print(merged_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/13/movies/one-...   
3  https://www.nytimes.com/2023/04/13/movies/the-...   
4  https://www.nytimes.com/2023/03/30/movies/a-th...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   
3  There’s not much Lennon music heard in this do...  The New York Times   
4  A mesmerizing Teyana Taylor stars in A.V. Rock...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;persons: Bracey, Lu

In [38]:
# Drop "byline.person" column
del merged_df["byline.person"]

In [39]:
# Delete duplicate rows and reset index
merged_df=merged_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('output\collected_data.csv', index=False)
 